In [1]:
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

The goal of this project is to project pitching scoring over the rest of a season in a Fantasy Baseball points league. In order to do this, I am going to run a linear regression on rate advanced pitching stats over the 2015-17 season as it relates to fantasy scoring. I am then going to run that against stats from the current 2018 season in order to find players that are underperforming fantasy wise from what their stats imply. 

In [16]:
# Read in the data from both sheets, drop the any null values. By how I got the data, this only includes
# starting pitchers with at least 50 IP
dfBasePitchingStats = pd.read_csv("Data/PitchingStats2015-17.csv")
dfBasePitchingStats = dfBasePitchingStats.dropna()
#Get rid of any pitcher that started less than 25 games to avoid incomplete stats
dfBasePitchingStats = dfBasePitchingStats.drop(dfBasePitchingStats[dfBasePitchingStats.GS < 25].index)

dfAdvPitchingStats = pd.read_csv("Data/AdvancedPitchingStats2015-17.csv")
dfAdvPitchingStats = dfAdvPitchingStats.dropna()

In [17]:
dfBasePitchingStats.head()

,Season,Name,Team,W,L,ERA,G,GS,CG,ShO,...,R,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid
0,2015,Zack Greinke,Dodgers,19,3,1.66,32,32,1,0,...,43,41,14,40,1,5,7,0,200,1943
2,2015,Jake Arrieta,Cubs,22,6,1.77,33,33,4,3,...,52,45,10,48,2,6,6,0,236,4153
4,2015,Clayton Kershaw,Dodgers,16,7,2.13,33,33,4,3,...,62,55,15,42,1,5,9,3,301,2036
5,2016,Kyle Hendricks,Cubs,16,8,2.15,30,30,2,1,...,53,45,15,43,3,8,5,0,169,12049
6,2017,Corey Kluber,Indians,18,4,2.25,29,29,5,3,...,56,51,21,36,2,5,4,0,265,2429


In [6]:
dfAdvPitchingStats.head()

,Season,Name,Team,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,...,LOB%,ERA-,FIP-,xFIP-,ERA,FIP,E-F,xFIP,SIERA,playerid
0,2016,Clayton Kershaw,Dodgers,10.39,0.66,15.64,0.48,31.6 %,2.0 %,29.6 %,...,80.0 %,42,45,56,1.69,1.80,-0.11,2.28,2.41,2036
1,2015,Clayton Kershaw,Dodgers,11.64,1.62,7.17,0.58,33.8 %,4.7 %,29.1 %,...,78.3 %,57,53,54,2.13,1.99,0.14,2.09,2.24,2036
2,2015,Jose Fernandez,Marlins,10.99,1.95,5.64,0.56,29.8 %,5.3 %,24.5 %,...,78.4 %,77,60,68,2.92,2.24,0.69,2.62,2.77,11530
3,2016,Jose Fernandez,Marlins,12.49,2.71,4.60,0.64,34.3 %,7.5 %,26.9 %,...,76.6 %,71,58,62,2.86,2.30,0.56,2.56,2.81,11530
4,2016,Noah Syndergaard,Mets,10.64,2.12,5.02,0.54,29.2 %,5.8 %,23.4 %,...,76.9 %,66,56,65,2.61,2.30,0.31,2.68,2.97,11762


In [9]:
# Enter your leagues scoring here  
W = 3
L = -3
G = 0
GS = 0
CG = 0
ShO = 0
SV = 5
HLD = 0
BS = 0
IP = 3
TBF = 0
H = -1
R = 0
ER = -2
HR = 0
BB = -1
IBB = 0
HBP = -1
WP = 0
BK = 0
SO = 1

In [18]:
#Calculate the 2015-2017 pitchers fantasy points
dfBasePitchingStats['FantasyPoints'] = (dfBasePitchingStats['W']*W) + (dfBasePitchingStats['L']*L) + (dfBasePitchingStats['G']*G) + (dfBasePitchingStats['GS']*GS) + (dfBasePitchingStats['CG']*CG) + (dfBasePitchingStats['CG']*CG) + (dfBasePitchingStats['ShO']*ShO) + (dfBasePitchingStats['SV']*SV) + (dfBasePitchingStats['HLD']*HLD) + (dfBasePitchingStats['BS']*BS) + (dfBasePitchingStats['IP']*IP) + (dfBasePitchingStats['TBF']*TBF) + (dfBasePitchingStats['H']*H) + (dfBasePitchingStats['R']*R) + (dfBasePitchingStats['ER']*ER) + (dfBasePitchingStats['HR']*HR) + (dfBasePitchingStats['BB']*BB) + (dfBasePitchingStats['IBB']*IBB) + (dfBasePitchingStats['HBP']*HBP) + (dfBasePitchingStats['WP']*WP) + (dfBasePitchingStats['BK']*BK) + (dfBasePitchingStats['SO']*SO)

In [19]:
dfBasePitchingStats.head()

,Season,Name,Team,W,L,ERA,G,GS,CG,ShO,...,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid,FantasyPoints
0,2015,Zack Greinke,Dodgers,19,3,1.66,32,32,1,0,...,41,14,40,1,5,7,0,200,1943,639.6
2,2015,Jake Arrieta,Cubs,22,6,1.77,33,33,4,3,...,45,10,48,2,6,6,0,236,4153,677.0
4,2015,Clayton Kershaw,Dodgers,16,7,2.13,33,33,4,3,...,55,15,42,1,5,9,3,301,2036,704.6
5,2016,Kyle Hendricks,Cubs,16,8,2.15,30,30,2,1,...,45,15,43,3,8,5,0,169,12049,474.0
6,2017,Corey Kluber,Indians,18,4,2.25,29,29,5,3,...,51,21,36,2,5,4,0,265,2429,632.6


In [20]:
#Copy the stuff we care about to a new temporary dataframe that we will use for merging
dfPitcherFantasy = dfBasePitchingStats[['Season', 'Name', 'FantasyPoints']].copy()
dfPitcherFantasy.head()

,Season,Name,FantasyPoints
0,2015,Zack Greinke,639.6
2,2015,Jake Arrieta,677.0
4,2015,Clayton Kershaw,704.6
5,2016,Kyle Hendricks,474.0
6,2017,Corey Kluber,632.6


In [60]:
# Merge the two dataframes using an inner join. This leaves us with a dataframe (dfStats) that has all of the advanced stats
# and fantasy points for each pitcher/season combo
dfStats = pd.merge(dfPitcherFantasy, dfAdvPitchingStats, how='inner', on=['Season', 'Name'])

#We don't need the team, season, name, or playerid; get rid of it
dfStats = dfStats.drop('Team', 1)
dfStats = dfStats.drop('playerid', 1)
dfStats = dfStats.drop('Name', 1)
dfStats = dfStats.drop('Season', 1)

#We have the league adjusted versions of these stats (ERA-, FIP-, xFIP-), so get rid of them
dfStats = dfStats.drop('ERA', 1)
dfStats = dfStats.drop('FIP', 1)
dfStats = dfStats.drop('xFIP', 1)

#This is just ERA-FIP, so get rid of it
dfStats = dfStats.drop('E-F', 1)
#Same for K-BB%, we have those stats already
dfStats = dfStats.drop('K-BB%', 1)
#Ditto for K/BB
dfStats = dfStats.drop('K/BB', 1)

#Format the columns from "XX%" to "0.XX" which require it
dfStats['K%'] = dfStats['K%'].str.rstrip('%').astype('float') / 100.0
dfStats['BB%'] = dfStats['BB%'].str.rstrip('%').astype('float') / 100.0
dfStats['LOB%'] = dfStats['LOB%'].str.rstrip('%').astype('float') / 100.0

#The linear regression hates names with % or / or numbers or -
dfStats = dfStats.rename(columns={'K%':'Kper'})
dfStats = dfStats.rename(columns={'BB%':'BBper'})
dfStats = dfStats.rename(columns={'LOB%':'LOBper'})
dfStats = dfStats.rename(columns={'K/9':'Kpernine'})
dfStats = dfStats.rename(columns={'BB/9':'BBpernine'})
dfStats = dfStats.rename(columns={'HR/9':'HRpernine'})
dfStats = dfStats.rename(columns={'ERA-':'ERAminus'})
dfStats = dfStats.rename(columns={'FIP-':'FIPminus'})
dfStats = dfStats.rename(columns={'xFIP-':'xFIPminus'})

dfStats.head()

,FantasyPoints,Kpernine,BBpernine,HRpernine,Kper,BBper,AVG,WHIP,BABIP,LOBper,ERAminus,FIPminus,xFIPminus,SIERA
0,639.6,8.08,1.62,0.57,0.237,0.047,0.185,0.84,0.229,0.865,44,73,83,3.27
1,677.0,9.28,1.89,0.39,0.271,0.055,0.184,0.86,0.246,0.800,45,61,68,2.75
2,704.6,11.64,1.62,0.58,0.338,0.047,0.193,0.88,0.281,0.783,57,53,54,2.24
3,474.0,8.09,2.06,0.72,0.229,0.058,0.207,0.98,0.252,0.814,51,77,86,3.68
4,632.6,11.71,1.59,0.93,0.341,0.046,0.192,0.87,0.267,0.826,50,57,57,2.68


In [53]:
dfStats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 275
Data columns (total 15 columns):
FantasyPoints    276 non-null float64
K/9              276 non-null float64
BB/9             276 non-null float64
K/BB             276 non-null float64
HR/9             276 non-null float64
K%               276 non-null float64
BB%              276 non-null float64
AVG              276 non-null float64
WHIP             276 non-null float64
BABIP            276 non-null float64
LOB%             276 non-null float64
ERA-             276 non-null int64
FIP-             276 non-null int64
xFIP-            276 non-null int64
SIERA            276 non-null float64
dtypes: float64(12), int64(3)
memory usage: 34.5 KB


Now we have a cleaned dataframe with the fantasy points from previous seasons and the stats that we want to run a linear regression on, perfect.

In [50]:
# This function performs a forward selection linear regression becuase we don't need all of the stats in our regression. I found
# it online. It passed the data science homework, so I'm gonna guess it works.
def forward_select(df, resp_str , maxk):
    
    remaining = set(df.columns)
    remaining.remove(resp_str)
    selected = []
    numselected = 1
    score_crnt, score_new = 0.0, 0.0
    while remaining and score_crnt == score_new:
        score_array = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(resp_str,' + '.join(selected + [candidate]))
            score = smf.ols(formula, df).fit().rsquared_adj
            score_array.append((score, candidate))
        score_array.sort()
        score_new, best_option = score_array.pop()
        if score_crnt < score_new and numselected <= maxk:
            remaining.remove(best_option)
            selected.append(best_option)
            score_crnt = score_new
            numselected += 1
    formula = "{} ~ {} + 1".format(resp_str,' + '.join(selected))
    model = smf.ols(formula, df).fit()
    return model

In [61]:
model = forward_select(dfStats, "FantasyPoints", 5)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          FantasyPoints   R-squared:                       0.895
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     459.8
Date:                Fri, 22 Jun 2018   Prob (F-statistic):          8.43e-130
Time:                        18:27:57   Log-Likelihood:                -1398.8
No. Observations:                 276   AIC:                             2810.
Df Residuals:                     270   BIC:                             2831.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    720.0961     73.905      9.744      0.000     574.593     865.599
WHIP        -251.4635     34.589     -7.270      0.000    -319.562    -183.365
FIPminus      -1.5804      0.253     -6.234      0.000      -2.079      -1.081
LOBper       563.2349     69.040      8.158      0.000     427.309     699.161
SIERA        -42.6882      7.336     -5.819      0.000     -57.132     -28.244
AVG         -706.1198    189.286     -3.730      0.000   -1078.784    -333.456
==============================================================================
Omnibus:                        5.073   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.079   Jarque-Bera (JB):                4.772
Skew:                          -0.307   Prob(JB):                       0.0920
Kurtosis:                       3.195   Cond. No.                     8.26e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

So if we run it with 5 outcomes, they are all very significant, and our top 5 are WHIP, FIP-, LOB%, SIERA, and AVG (opponent AVG). Let's try running it with more and see how many are significant.

In [62]:
model = forward_select(dfStats, "FantasyPoints", 10)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          FantasyPoints   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     251.4
Date:                Fri, 22 Jun 2018   Prob (F-statistic):          5.29e-129
Time:                        18:29:37   Log-Likelihood:                -1385.3
No. Observations:                 276   AIC:                             2793.
Df Residuals:                     265   BIC:                             2832.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    653.4456    119.365      5.474      0.000     418.422     888.469
WHIP         220.7475    186.916      1.181      0.239    -147.283     588.777
FIPminus      -0.8474      0.450     -1.883      0.061      -1.733       0.038
LOBper       264.6406    114.479      2.312      0.022      39.237     490.044
SIERA        -71.9838     13.506     -5.330      0.000     -98.577     -45.390
AVG        -1685.8812    892.830     -1.888      0.060   -3443.824      72.062
ERAminus      -1.3652      0.468     -2.916      0.004      -2.287      -0.443
xFIPminus      1.5058      0.556      2.710      0.007       0.412       2.600
Kper        4334.0872    998.715      4.340      0.000    2367.662    6300.512
Kpernine    -109.4758     26.133     -4.189      0.000    -160.930     -58.021
BBper      -1103.2930    705.030     -1.565      0.119   -2491.466     284.880
==============================================================================
Omnibus:                        5.071   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.079   Jarque-Bera (JB):                4.765
Skew:                          -0.305   Prob(JB):                       0.0923
Kurtosis:                       3.203   Cond. No.                     9.28e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.28e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Our Adjusted R-Squared actually got better, but BB/9 has too high of a p-value. It seems like using the top 9 is the way to go since they all have p-values of ~0.

In [63]:
model = forward_select(dfStats, "FantasyPoints", 9)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          FantasyPoints   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     277.6
Date:                Fri, 22 Jun 2018   Prob (F-statistic):          1.04e-129
Time:                        18:30:57   Log-Likelihood:                -1386.6
No. Observations:                 276   AIC:                             2793.
Df Residuals:                     266   BIC:                             2829.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    585.4185    111.470      5.252      0.000     365.943     804.894
WHIP         -55.5162     61.581     -0.902      0.368    -176.765      65.733
FIPminus      -0.8758      0.451     -1.943      0.053      -1.763       0.012
LOBper       255.0966    114.627      2.225      0.027      29.405     480.788
SIERA        -72.3003     13.542     -5.339      0.000     -98.963     -45.638
AVG         -364.1856    290.280     -1.255      0.211    -935.725     207.353
ERAminus      -1.3065      0.468     -2.792      0.006      -2.228      -0.385
xFIPminus      1.5812      0.555      2.848      0.005       0.488       2.674
Kper        3862.0448    954.661      4.045      0.000    1982.391    5741.698
Kpernine     -95.9344     24.726     -3.880      0.000    -144.617     -47.252
==============================================================================
Omnibus:                        4.817   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.090   Jarque-Bera (JB):                4.505
Skew:                          -0.296   Prob(JB):                        0.105
Kurtosis:                       3.201   Cond. No.                     7.21e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.21e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

So now we have our model.

Fantasy Points = (WHIP $*$ -55.5)+(FIP- $*$ -0.88)+(LOB% $*$ 255.1)+(SIERA $*$ -72.3)+(AVG $*$ -365.2)+(ERA- $*$ -1.3)+(xFIP- $*$ 1.6)+(K% $*$ 3862)+(K/9 $*$ -95.9)+585.42

Let's take a look at how this projects over the current season. 

In [76]:
# Read in the data from both sheets, drop the any null values. By how I got the data, this only includes
# starting pitchers with at least 50 IP
dfBasePitchingStats2018 = pd.read_csv("Data/PitchingStats2018.csv")
dfBasePitchingStats2018 = dfBasePitchingStats2018.dropna()

dfAdvPitchingStats2018 = pd.read_csv("Data/AdvancedPitchingStats2018.csv")
dfAdvPitchingStats2018 = dfAdvPitchingStats2018.dropna()

In [77]:
# This calculates what each player is on pace for over 32 games started, just the normal calculation weighted by 
# 32/GS for the season thus far
dfBasePitchingStats2018['PaceFantasyPoints'] = ((dfBasePitchingStats2018['W']*W) + (dfBasePitchingStats2018['L']*L) + (dfBasePitchingStats2018['G']*G) + (dfBasePitchingStats2018['GS']*GS) + (dfBasePitchingStats2018['CG']*CG) + (dfBasePitchingStats2018['CG']*CG) + (dfBasePitchingStats2018['ShO']*ShO) + (dfBasePitchingStats2018['SV']*SV) + (dfBasePitchingStats2018['HLD']*HLD) + (dfBasePitchingStats2018['BS']*BS) + (dfBasePitchingStats2018['IP']*IP) + (dfBasePitchingStats2018['TBF']*TBF) + (dfBasePitchingStats2018['H']*H) + (dfBasePitchingStats2018['R']*R) + (dfBasePitchingStats2018['ER']*ER) + (dfBasePitchingStats2018['HR']*HR) + (dfBasePitchingStats2018['BB']*BB) + (dfBasePitchingStats2018['IBB']*IBB) + (dfBasePitchingStats2018['HBP']*HBP) + (dfBasePitchingStats2018['WP']*WP) + (dfBasePitchingStats2018['BK']*BK) + (dfBasePitchingStats2018['SO']*SO))*(32/dfBasePitchingStats2018['GS'])

In [78]:
dfBasePitchingStats2018.head()

,Season,Name,Team,W,L,ERA,G,GS,CG,ShO,...,ER,HR,BB,IBB,HBP,WP,BK,SO,playerid,PaceFantasyPoints
0,2018,Jacob deGrom,Mets,5,2,1.51,15,15,0,0,...,16,4,24,2,3,0,0,120,10954,608.64
1,2018,Justin Verlander,Astros,9,2,1.60,16,16,1,1,...,19,9,21,0,5,2,0,130,8700,692.00
2,2018,Max Scherzer,Nationals,10,3,2.09,16,16,1,1,...,25,10,24,1,5,3,0,161,3137,715.20
3,2018,Corey Kluber,Indians,11,3,2.10,16,16,1,0,...,26,14,12,0,1,2,0,113,2429,655.20
4,2018,Jon Lester,Cubs,9,2,2.10,15,15,0,0,...,21,10,31,1,5,1,0,70,4930,460.80


In [80]:
#Copy the stuff we care about to a new temporary dataframe that we will use for merging
dfPitcherFantasy18 = dfBasePitchingStats2018[['Season', 'Name', 'PaceFantasyPoints']].copy()
dfPitcherFantasy18.head()

,Season,Name,PaceFantasyPoints
0,2018,Jacob deGrom,608.64
1,2018,Justin Verlander,692.00
2,2018,Max Scherzer,715.20
3,2018,Corey Kluber,655.20
4,2018,Jon Lester,460.80


In [81]:
# Merge the two dataframes using an inner join. This leaves us with a dataframe (dfStats) that has all of the advanced stats
# and fantasy points for each pitcher/season combo
dfStats18 = pd.merge(dfPitcherFantasy18, dfAdvPitchingStats2018, how='inner', on=['Season', 'Name'])

#We don't need the season or playerid; get rid of it
dfStats18 = dfStats18.drop('Season', 1)
dfStats18 = dfStats18.drop('playerid', 1)

#We have the league adjusted versions of these stats (ERA-, FIP-, xFIP-), so get rid of them
dfStats18 = dfStats18.drop('ERA', 1)
dfStats18 = dfStats18.drop('FIP', 1)
dfStats18 = dfStats18.drop('xFIP', 1)

#This is just ERA-FIP, so get rid of it
dfStats18 = dfStats18.drop('E-F', 1)
#Same for K-BB%, we have those stats already
dfStats18 = dfStats18.drop('K-BB%', 1)
#Ditto for K/BB
dfStats18 = dfStats18.drop('K/BB', 1)

#Format the columns from "XX%" to "0.XX" which require it
dfStats18['K%'] = dfStats18['K%'].str.rstrip('%').astype('float') / 100.0
dfStats18['BB%'] = dfStats18['BB%'].str.rstrip('%').astype('float') / 100.0
dfStats18['LOB%'] = dfStats18['LOB%'].str.rstrip('%').astype('float') / 100.0

dfStats18.head()

,Name,PaceFantasyPoints,Team,K/9,BB/9,HR/9,K%,BB%,AVG,WHIP,BABIP,LOB%,ERA-,FIP-,xFIP-,SIERA
0,Jacob deGrom,608.64,Mets,11.33,2.27,0.38,0.321,0.064,0.202,0.99,0.296,0.853,40,49,61,2.76
1,Justin Verlander,692.00,Astros,10.93,1.77,0.76,0.320,0.052,0.163,0.78,0.220,0.889,40,61,85,2.95
2,Max Scherzer,715.20,Nationals,13.46,2.01,0.84,0.386,0.058,0.173,0.85,0.263,0.829,52,53,63,2.21
3,Corey Kluber,655.20,Indians,9.11,0.97,1.13,0.272,0.029,0.194,0.81,0.232,0.882,49,74,66,2.86
4,Jon Lester,460.80,Cubs,7.00,3.10,1.00,0.193,0.085,0.205,1.09,0.231,0.876,52,104,112,4.53


In [83]:
dfStats18['ProjFantasyPoints'] = (dfStats18['WHIP']*-55.5)+(dfStats18['FIP-']*-0.88)+(dfStats18['LOB%']*255.1)+(dfStats18['SIERA']*-72.3)+(dfStats18['AVG']*-365.2)+(dfStats18['ERA-']*-1.3)+(dfStats18['xFIP-']*1.6)+(dfStats18['K%']*3862)+(dfStats18['K/9']*-95.9)+585.42
dfStats18 = dfStats18[['Name', 'Team', 'PaceFantasyPoints', 'ProjFantasyPoints']].copy()
dfStats18.head()


,Name,Team,PaceFantasyPoints,ProjFantasyPoints
0,Jacob deGrom,Mets,608.64,630.3919
1,Justin Verlander,Astros,692.00,714.0743
2,Max Scherzer,Nationals,715.20,713.2383
3,Corey Kluber,Indians,655.20,641.4314
4,Jon Lester,Cubs,460.80,440.1536


Those look... pretty damn good. Let's look at the top 25 projected fantasy starting pitchers.

In [85]:
dfStats18.sort_values(by=['ProjFantasyPoints'], ascending=False)

,Name,Team,PaceFantasyPoints,ProjFantasyPoints
1,Justin Verlander,Astros,692.000000,714.0743
2,Max Scherzer,Nationals,715.200000,713.2383
11,Gerrit Cole,Astros,658.346667,668.5363
3,Corey Kluber,Indians,655.200000,641.4314
0,Jacob deGrom,Mets,608.640000,630.3919
6,Luis Severino,Yankees,613.200000,595.8679
16,Chris Sale,Red Sox,572.000000,591.9317
7,Ross Stripling,Dodgers,524.800000,584.4125
12,Walker Buehler,Dodgers,481.066667,525.4984
10,Aaron Nola,Phillies,523.306667,520.0194


Seeing the best guys is helpful. But we're looking for value. So let's find guys who are underperforming.

In [86]:
dfStats18['Points Diff'] = dfStats18['ProjFantasyPoints'] - dfStats18['PaceFantasyPoints']
dfStats18.sort_values(by=['Points Diff'], ascending=False)

,Name,Team,PaceFantasyPoints,ProjFantasyPoints,Points Diff
19,Junior Guerra,Brewers,351.015385,426.0487,75.033315
62,Nick Pivetta,Phillies,333.440000,396.7979,63.357900
7,Ross Stripling,Dodgers,524.800000,584.4125,59.612500
74,Jose Urena,Marlins,248.000000,304.9192,56.919200
13,Jack Flaherty,Cardinals,425.244444,475.4444,50.199956
79,Doug Fister,Rangers,149.333333,194.7218,45.388467
12,Walker Buehler,Dodgers,481.066667,525.4984,44.431733
58,Caleb Smith,Marlins,313.600000,356.6460,43.046000
41,Matt Boyd,Tigers,313.828571,349.7841,35.955529
59,Jameson Taillon,Pirates,299.946667,332.2972,32.350533


And now we can see the real value. These are the pitchers we want to target for trades, and also the pitchers we want to get rid of ASAP. 